This is a boiler plate code for multi-fold SVM, RF training and testing which has been attempted to be used for Hans' Jiang Conrad similarity values

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
import scipy.stats as stats
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from itertools import combinations
from math import comb
from sklearn.ensemble import RandomForestClassifier

# Determine Differences

In [3]:
wordSim353AnnotDF_New_Merged_DF = pd.read_csv('../data/wordsim353_annotation_task_qnodes_kartik_20210629+IDs+jiang_conrath_sim.tsv', sep='\t')

In [4]:
wordsimLat = pd.read_csv('../data/wordsim353_all_embeddings_with_retrofits.csv')

In [6]:
wordSim353AnnotDF_New_Merged_DF = wordSim353AnnotDF_New_Merged_DF.set_index(['Word 1', 'Word 2']).join(wordsimLat.set_index(['Word 1', 'Word 2']),rsuffix='_right')

In [9]:
wordSim353AnnotDF_New_Merged_DF = wordSim353AnnotDF_New_Merged_DF.reset_index()

In [10]:
wordSim353AnnotDF_New_Merged_DF.head()

,Word 1,Word 2,ID,word1_kg_id,word1_kg_label,word1_kg_description,word2_kg_id,word2_kg_label,word2_kg_description,Human (Mean),...,concat_19k_v2_word1_new,concat_19k_v2_word2_new,concat_19k_v2_old_cosSim,concat_19k_v2_2_new_cosSim,concat_probase_19k_v2_word1_old,concat_probase_19k_v2_word2_old,concat_probase_19k_v2_word1_new,concat_probase_19k_v2_word2_new,concat_probase_19k_v2_old_cosSim,concat_probase_19k_v2_2_new_cosSim
0,Arafat,peace,8,Q34211,Yasser Arafat,former Palestinian President,Q454,peace,state of harmony characterized by lack of viol...,2.1250,...,"[0.006720443092017323, 0.008331487679612525, 0...","[-0.004515859576248157, -0.01172366887295375, ...",3.244539,3.175157,"[0.006720440291834372, 0.008331484208161396, 0...","[-0.005011610937734838, -0.013383541550578623,...","[0.006720440291834372, 0.008331484208161396, 0...","[-0.00497129831746384, -0.01325426749683971, -...",3.244539,3.240270
1,Arafat,terror,9,Q34211,Yasser Arafat,former Palestinian President,Q13648784,terror,policy of political repression and violence,3.0625,...,"[0.006720443092017323, 0.008331487679612525, 0...","[-0.004862257088069111, 0.014077170590128979, ...",3.085083,3.013958,"[0.006720440291834372, 0.008331484208161396, 0...","[-0.005805280438126139, 0.015115419696749257, ...","[0.006720440291834372, 0.008331484208161396, 0...","[-0.004862255062129871, 0.014077164724644747, ...",3.085083,3.013958
2,FBI,fingerprint,109,Q8333,Federal Bureau of Investigation,United States law enforcement agency,Q178022,fingerprint,biometric identifier,4.0625,...,"[0.006466897059812569, 0.014694837542908411, -...","[0.01523113309590259, 0.005271417380511587, -0...",2.823093,2.745230,"[0.006466894365273728, 0.014694831420063071, -...","[0.013922101136130544, 0.0023911842131386003, ...","[0.006466894365273728, 0.014694831420063071, -...","[0.013814564227839972, 0.0024352598952305004, ...",2.823093,2.816176
3,FBI,investigation,110,Q8333,Federal Bureau of Investigation,United States law enforcement agency,Q21004260,inquiry,process that has the aim of augmenting knowled...,5.0625,...,"[0.006466897059812569, 0.014694837542908411, -...","[0.013016090623183913, 0.01114268771757639, -0...",2.920986,2.886161,"[0.006466894365273728, 0.014694831420063071, -...","[0.01327496060589727, 0.010965634890657862, -0...","[0.006466894365273728, 0.014694831420063071, -...","[0.013242882546106787, 0.010953106196386554, -...",2.920986,2.919120
4,Harvard,Yale,137,Q13371,Harvard University,"private research university in Cambridge, Mass...",Q49112,Yale University,"private research university in New Haven, Conn...",4.8750,...,"[0.01322626159341165, 0.005047111630696947, -0...","[0.010410829393995813, 0.014776997996557633, -...",1.707655,1.707655,"[0.013226256082472592, 0.005047109527735016, -...","[0.010410825056152807, 0.014776991839478788, -...","[0.013226256082472592, 0.005047109527735016, -...","[0.010410825056152807, 0.014776991839478788, -...",1.707655,1.707655


In [11]:
[wordSim353AnnotDF_New_Merged_DF[col].isna().sum() for col in ['Human (Mean)', 'complex', 'transe', 'text', 'class', 'jc_sim']]

[0, 1, 1, 19, 4, 0]

In [12]:
for col in ['complex', 'transe', 'text', 'class', 'jc_sim']:
    wordSim353AnnotDF_New_Merged_DF[col] = wordSim353AnnotDF_New_Merged_DF[col].fillna(-1)

In [5]:
# def determineFarNessOfOld(val1):
#     if 0.000000 <= val1 < 0.496332:
#         return 0
#     else:
#         return 1
# def determineFarNessOfNew(val1):
#     if 0 <= val1 < 0.570044:
#         return 0
#     else:
#         return 1

# wordSim353AnnotDF_New_Merged_DF['transe_19k_old_cosSim_Diff_Cat'] = wordSim353AnnotDF_New_Merged_DF['transe_19k_old_cosSim_Diff'].apply(determineFarNessOfOld)
# wordSim353AnnotDF_New_Merged_DF['transe_19k_1_new_cosSim_Diff_Cat'] = wordSim353AnnotDF_New_Merged_DF['transe_19k_1_new_cosSim_Diff'].apply(determineFarNessOfNew)

In [6]:
# wordSim353AnnotDF_New_Merged_DF = wordSim353AnnotDF_New_Merged_DF[['Word 1', 'Word 2', 'Avg', 'transe_19k_old_cosSim', 'transe_19k_1_new_cosSim', 'transe_19k_old_cosSim_Diff', 'transe_19k_1_new_cosSim_Diff', 'transe_19k_old_cosSim_Diff_Cat', 'transe_19k_1_new_cosSim_Diff_Cat']]

In [7]:
# wordSim353AnnotDF_New_Merged_DF

In [8]:
# len(wordSim353AnnotDF_New_Merged_DF[(wordSim353AnnotDF_New_Merged_DF['transe_19k_old_cosSim_Diff_Cat'] == 0) & (wordSim353AnnotDF_New_Merged_DF['transe_19k_1_new_cosSim_Diff_Cat'] == 0)])

In [9]:
# wordSim353AnnotDF_New_Merged_DF = wordSim353AnnotDF_New_Merged_DF[]

In [10]:
# wordSim353AnnotDF_New_Merged_DF[(wordSim353AnnotDF_New_Merged_DF['transe_19k_old_cosSim_Diff_Cat'] == 0) & (wordSim353AnnotDF_New_Merged_DF['transe_19k_1_new_cosSim_Diff_Cat'] == 0)].to_csv('../data/transEmb/good+good.csv',index=None)

In [11]:
# wordSim353AnnotDF_New_Merged_DF[(wordSim353AnnotDF_New_Merged_DF['transe_19k_old_cosSim_Diff_Cat'] == 0) & (wordSim353AnnotDF_New_Merged_DF['transe_19k_1_new_cosSim_Diff_Cat'] == 1)].to_csv('../data/transEmb/good+bad.csv',index=None)

In [12]:
# wordSim353AnnotDF_New_Merged_DF[(wordSim353AnnotDF_New_Merged_DF['transe_19k_old_cosSim_Diff_Cat'] == 1) & (wordSim353AnnotDF_New_Merged_DF['transe_19k_1_new_cosSim_Diff_Cat'] == 0)].to_csv('../data/transEmb/bad+good.csv',index=None)

In [13]:
# wordSim353AnnotDF_New_Merged_DF[(wordSim353AnnotDF_New_Merged_DF['transe_19k_old_cosSim_Diff_Cat'] == 1) & (wordSim353AnnotDF_New_Merged_DF['transe_19k_1_new_cosSim_Diff_Cat'] == 1)].to_csv('../data/transEmb/bad+bad.csv',index=None)

# SVM Models

In [14]:
wordSim353AnnotDF_New_Merged_DF = pd.read_csv('../data/wordsim353_annotation_task_qnodes_kartik_20210629+IDs+jiang_conrath_sim.tsv', sep='\t')

In [13]:
wordSim353AnnotDF_New_Merged_DF.head()

,Word 1,Word 2,ID,word1_kg_id,word1_kg_label,word1_kg_description,word2_kg_id,word2_kg_label,word2_kg_description,Human (Mean),...,concat_19k_v2_word1_new,concat_19k_v2_word2_new,concat_19k_v2_old_cosSim,concat_19k_v2_2_new_cosSim,concat_probase_19k_v2_word1_old,concat_probase_19k_v2_word2_old,concat_probase_19k_v2_word1_new,concat_probase_19k_v2_word2_new,concat_probase_19k_v2_old_cosSim,concat_probase_19k_v2_2_new_cosSim
0,Arafat,peace,8,Q34211,Yasser Arafat,former Palestinian President,Q454,peace,state of harmony characterized by lack of viol...,2.1250,...,"[0.006720443092017323, 0.008331487679612525, 0...","[-0.004515859576248157, -0.01172366887295375, ...",3.244539,3.175157,"[0.006720440291834372, 0.008331484208161396, 0...","[-0.005011610937734838, -0.013383541550578623,...","[0.006720440291834372, 0.008331484208161396, 0...","[-0.00497129831746384, -0.01325426749683971, -...",3.244539,3.240270
1,Arafat,terror,9,Q34211,Yasser Arafat,former Palestinian President,Q13648784,terror,policy of political repression and violence,3.0625,...,"[0.006720443092017323, 0.008331487679612525, 0...","[-0.004862257088069111, 0.014077170590128979, ...",3.085083,3.013958,"[0.006720440291834372, 0.008331484208161396, 0...","[-0.005805280438126139, 0.015115419696749257, ...","[0.006720440291834372, 0.008331484208161396, 0...","[-0.004862255062129871, 0.014077164724644747, ...",3.085083,3.013958
2,FBI,fingerprint,109,Q8333,Federal Bureau of Investigation,United States law enforcement agency,Q178022,fingerprint,biometric identifier,4.0625,...,"[0.006466897059812569, 0.014694837542908411, -...","[0.01523113309590259, 0.005271417380511587, -0...",2.823093,2.745230,"[0.006466894365273728, 0.014694831420063071, -...","[0.013922101136130544, 0.0023911842131386003, ...","[0.006466894365273728, 0.014694831420063071, -...","[0.013814564227839972, 0.0024352598952305004, ...",2.823093,2.816176
3,FBI,investigation,110,Q8333,Federal Bureau of Investigation,United States law enforcement agency,Q21004260,inquiry,process that has the aim of augmenting knowled...,5.0625,...,"[0.006466897059812569, 0.014694837542908411, -...","[0.013016090623183913, 0.01114268771757639, -0...",2.920986,2.886161,"[0.006466894365273728, 0.014694831420063071, -...","[0.01327496060589727, 0.010965634890657862, -0...","[0.006466894365273728, 0.014694831420063071, -...","[0.013242882546106787, 0.010953106196386554, -...",2.920986,2.919120
4,Harvard,Yale,137,Q13371,Harvard University,"private research university in Cambridge, Mass...",Q49112,Yale University,"private research university in New Haven, Conn...",4.8750,...,"[0.01322626159341165, 0.005047111630696947, -0...","[0.010410829393995813, 0.014776997996557633, -...",1.707655,1.707655,"[0.013226256082472592, 0.005047109527735016, -...","[0.010410825056152807, 0.014776991839478788, -...","[0.013226256082472592, 0.005047109527735016, -...","[0.010410825056152807, 0.014776991839478788, -...",1.707655,1.707655


In [14]:
wordSim353AnnotDF_New_Merged_DF.columns.to_list()

['Word 1',
 'Word 2',
 'ID',
 'word1_kg_id',
 'word1_kg_label',
 'word1_kg_description',
 'word2_kg_id',
 'word2_kg_label',
 'word2_kg_description',
 'Human (Mean)',
 'complex',
 'transe',
 'text',
 'class',
 'jc_sim',
 'ID_right',
 'H_Sim',
 'H_Dim',
 'F_Sim',
 'F_Dim',
 'N_Sim',
 'N_Dim',
 'D_Sim',
 'D_Dim',
 'P_Sim',
 'P_Dim',
 'Avg',
 'Stdev',
 'H_orig',
 'H_reversed',
 'word1_kg_id_right',
 'word2_kg_id_right',
 'category',
 'text_7props_19k_word1_old',
 'text_7props_19k_word2_old',
 'text_7props_19k_word1_new',
 'text_7props_19k_word2_new',
 'text_7props_19k_old_cosSim',
 'text_7props_19k_2_new_cosSim',
 'text_2props_19k_word1_old',
 'text_2props_19k_word2_old',
 'text_2props_19k_word1_new',
 'text_2props_19k_word2_new',
 'text_2props_19k_old_cosSim',
 'text_2props_19k_2_new_cosSim',
 'complex_19k_word1_old',
 'complex_19k_word2_old',
 'complex_19k_word1_new',
 'complex_19k_word2_new',
 'complex_19k_old_cosSim',
 'complex_19k_1_new_cosSim',
 'transe_19k_word1_old',
 'transe_19k_w

In [16]:
# cosine_similarity(np.array([float(val1) for val1 in wordSim353AnnotDF_New_Merged_DF.iloc[0]['concat_19k_v2_word1_new'][1:-1].split(",")]).reshape(1,-1), np.array([float(val1) for val1 in wordSim353AnnotDF_New_Merged_DF.iloc[0]['concat_19k_v2_word2_new'][1:-1].split(",")]).reshape(1,-1))

In [17]:
wordSim353AnnotDF_New_Merged_DF.columns

Index(['ID', 'Word 1', 'word1_kg_id', 'word1_kg_label', 'word1_kg_description',
       'Word 2', 'word2_kg_id', 'word2_kg_label', 'word2_kg_description',
       'Human (Mean)', 'complex', 'transe', 'text', 'class', 'jc_sim'],
      dtype='object')

In [15]:
from sklearn.model_selection import KFold

import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

# from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
import seaborn as sns
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt 

In [16]:
# def trainAndGenerateAccuracies(wordSim353AnnotDF_New_Merged_DF, colList):
#     X = wordSim353AnnotDF_New_Merged_DF[colList]
#     Y = wordSim353AnnotDF_New_Merged_DF['Avg']

#     N_SPLITS = 10

#     skf = KFold(n_splits=N_SPLITS, random_state=19, shuffle=True)
#     X_train_splits, X_test_splits, Y_train_splits, Y_test_splits = [], [], [], []
#     for train_index, test_index in skf.split(X, Y):
#         X_train_splits.append(X.iloc[train_index])
#         X_test_splits.append(X.iloc[test_index])
#         Y_train_splits.append(Y.iloc[train_index])
#         Y_test_splits.append(Y.iloc[test_index])

#     preds = []
#     for X_train1, Y_train1, X_test1, Y_test1 in zip(X_train_splits, Y_train_splits, X_test_splits, Y_test_splits):
#         clf = make_pipeline(StandardScaler(), SVR(gamma='auto'))
#         clf.fit(X_train1, Y_train1)
#         preds.append(clf.predict(X_test1))

#     tempVals = []å

#     acc = 0
#     for pred, Y_test1 in zip(preds, Y_test_splits):
#         acc += mean_squared_error(pred, Y_test1)

#     tempVals.append(acc/N_SPLITS)

#     for col in colList:
#         preds = []
#         for X_train1, Y_train1, X_test1, Y_test1 in zip(X_train_splits, Y_train_splits, X_test_splits, Y_test_splits):
#             clf1 = make_pipeline(StandardScaler(), SVR(gamma='auto'))
#             clf1.fit(X_train1[[col]], Y_train1)
#             preds.append(clf1.predict(X_test1[[col]]))
#         acc = 0
#         for pred, Y_test1 in zip(preds, Y_test_splits):
#             acc += mean_squared_error(pred, Y_test1)
#         tempVals.append(acc/N_SPLITS)
#     return tempVals

# def compareEmbeddings(wordSim353AnnotDF_New_Merged_DF, oldColList, newColList, indNames):
#     tempVals1 = trainAndGenerateAccuracies(wordSim353AnnotDF_New_Merged_DF, oldColList)
#     tempVals2 = trainAndGenerateAccuracies(wordSim353AnnotDF_New_Merged_DF, newColList)
#     summ = (pd.DataFrame(list(zip(tempVals1, tempVals2)), index = ['Combined'] + indNames, columns = ['MSE of old embeddings compared to annotated category', 'MSE of new embeddings compared to annotated category']) * 100)
#     summ['Increase'] = summ['MSE of new embeddings compared to annotated category'] - summ['MSE of old embeddings compared to annotated category']
#     return summ.sort_values(by=['Increase'], ascending=False)


In [90]:
def trainAndFindAccuracy(wordSim353AnnotDF_New_Merged_DF, colList):
    X = wordSim353AnnotDF_New_Merged_DF[colList]
    Y = wordSim353AnnotDF_New_Merged_DF['Avg']

    N_SPLITS = 10

    skf = KFold(n_splits=N_SPLITS, random_state=19, shuffle=True)
    X_train_splits, X_test_splits, Y_train_splits, Y_test_splits = [], [], [], []
    for train_index, test_index in skf.split(X, Y):
        X_train_splits.append(X.iloc[train_index])
        X_test_splits.append(X.iloc[test_index])
        Y_train_splits.append(Y.iloc[train_index])
        Y_test_splits.append(Y.iloc[test_index])

    preds = []
    for X_train1, Y_train1, X_test1, Y_test1 in zip(X_train_splits, Y_train_splits, X_test_splits, Y_test_splits):
        clf = make_pipeline(StandardScaler(), SVR(gamma='auto'))
        clf.fit(X_train1, Y_train1)
        preds.append(clf.predict(X_test1))

    tempVals = []

    acc = 0
    for pred, Y_test1 in zip(preds, Y_test_splits):
        acc += mean_squared_error(pred, Y_test1)

    return acc/N_SPLITS

In [71]:
len(wordSim353AnnotDF_New_Merged_DF)

349

In [18]:
# def trainAndFindAccuracyClassification(wordSim353AnnotDF_New_Merged_DF, colList):
#     X = wordSim353AnnotDF_New_Merged_DF[colList]
#     Y = wordSim353AnnotDF_New_Merged_DF['category']

#     N_SPLITS = 10

#     skf = StratifiedKFold(n_splits=N_SPLITS, random_state=19, shuffle=True)
#     X_train_splits, X_test_splits, Y_train_splits, Y_test_splits = [], [], [], []
#     for train_index, test_index in skf.split(X, Y):
#         X_train_splits.append(X.iloc[train_index])
#         X_test_splits.append(X.iloc[test_index])
#         Y_train_splits.append(Y.iloc[train_index])
#         Y_test_splits.append(Y.iloc[test_index])

#     preds = []
#     for X_train1, Y_train1, X_test1, Y_test1 in zip(X_train_splits, Y_train_splits, X_test_splits, Y_test_splits):
#         clf = make_pipeline(StandardScaler(), SVC(gamma='auto', random_state=100))
#         clf.fit(X_train1, Y_train1)
#         preds.append(clf.predict(X_test1))

#     tempVals = []

#     acc = 0
#     for pred, Y_test1 in zip(preds, Y_test_splits):
#         acc += accuracy_score(pred, Y_test1)

#     return acc/N_SPLITS

In [19]:
# def trainAndFindAccuracyRF(wordSim353AnnotDF_New_Merged_DF, colList):
#     X = wordSim353AnnotDF_New_Merged_DF[colList]
#     Y = wordSim353AnnotDF_New_Merged_DF['category']

#     N_SPLITS = 10

#     skf = StratifiedKFold(n_splits=N_SPLITS, random_state=19, shuffle=True)
#     X_train_splits, X_test_splits, Y_train_splits, Y_test_splits = [], [], [], []
#     for train_index, test_index in skf.split(X, Y):
#         X_train_splits.append(X.iloc[train_index])
#         X_test_splits.append(X.iloc[test_index])
#         Y_train_splits.append(Y.iloc[train_index])
#         Y_test_splits.append(Y.iloc[test_index])

#     preds = []
#     for X_train1, Y_train1, X_test1, Y_test1 in zip(X_train_splits, Y_train_splits, X_test_splits, Y_test_splits):
#         clf = RandomForestClassifier(max_depth=max(2 * len(colList) // 3,3), random_state=100)
#         clf.fit(X_train1, Y_train1)
#         preds.append(clf.predict(X_test1))

#     tempVals = []

#     acc = 0
#     for pred, Y_test1 in zip(preds, Y_test_splits):
#         acc += accuracy_score(pred, Y_test1)

#     return acc/N_SPLITS

In [51]:
colList = ['complex', 'transe', 'text', 'class', 'jc_sim']
indNames = ['Complex', 'Transe', 'Text', 'Class', 'JC_Sim']
colPairs = list(zip(colList, indNames))
trainAndFindAccuracy(wordSim353AnnotDF_New_Merged_DF, colList)

0.016446209419554415

## Regression

In [38]:
wordSim353AnnotDF_New_Merged_DF['Avg1'] = wordSim353AnnotDF_New_Merged_DF['Avg']

In [39]:
wordSim353AnnotDF_New_Merged_DF['Avg'] = wordSim353AnnotDF_New_Merged_DF['Avg'].apply(lambda p: 1-(p - 1)/4)

In [49]:
wordSim353AnnotDF_New_Merged_DF['Avg'].describe()

count    349.000000
mean     0.490867  
std      0.189153  
min      0.250000  
25%      0.350000  
50%      0.500000  
75%      0.600000  
max      1.000000  
Name: Avg, dtype: float64

In [91]:
from joblib import Parallel, delayed
def generateCombAccuracies(oldColPairs, r):
    accs = []
    for comb in tqdm(combinations(oldColPairs, r)):
        assert len(comb) == r
        oldAcc = trainAndFindAccuracy(wordSim353AnnotDF_New_Merged_DF, [col[0] for col in comb])
#         newAcc = trainAndFindAccuracy(wordSim353AnnotDF_New_Merged_DF, [colMappers[col[0]] for col in comb])
        indName = " & ".join([col[1] for col in comb])
        accs.append((indName, oldAcc))
    return accs
    
results = Parallel(n_jobs=14)(delayed(generateCombAccuracies)(colPairs, i) for i in tqdm(range(1,len(colPairs)+1)))
print(results)

  0%|          | 0/5 [00:00<?, ?it/s]

[[('Complex', 0.02321940095771151), ('Transe', 0.026212630341633864), ('Text', 0.02245616090133273), ('Class', 0.01936172956841233), ('JC_Sim', 0.01933705421406063)], [('Complex & Transe', 0.023498903093071868), ('Complex & Text', 0.020288804874437343), ('Complex & Class', 0.019280969582380746), ('Complex & JC_Sim', 0.018391197762239037), ('Transe & Text', 0.02164415251389583), ('Transe & Class', 0.019243961232510827), ('Transe & JC_Sim', 0.01915285516489467), ('Text & Class', 0.017471550608673957), ('Text & JC_Sim', 0.017272773472134774), ('Class & JC_Sim', 0.018168085523900198)], [('Complex & Transe & Text', 0.020668807932076618), ('Complex & Transe & Class', 0.018976450797180543), ('Complex & Transe & JC_Sim', 0.01856637416827929), ('Complex & Text & Class', 0.016934178158318845), ('Complex & Text & JC_Sim', 0.017271869864320485), ('Complex & Class & JC_Sim', 0.018330482575486425), ('Transe & Text & Class', 0.016909041915910205), ('Transe & Text & JC_Sim', 0.017556257532620998), ('T

In [92]:
# from joblib import Parallel, delayed
# def generateCombAccuracies(oldColPairs, r):
#     accs = []
#     for comb in tqdm(combinations(oldColPairs, r)):
#         assert len(comb) == r
#         oldAcc = trainAndFindAccuracyRF(wordSim353AnnotDF_New_Merged_DF, [col[0] for col in comb])
#         newAcc = trainAndFindAccuracyRF(wordSim353AnnotDF_New_Merged_DF, [colMappers[col[0]] for col in comb])
#         indName = " & ".join([col[1] for col in comb])
#         accs.append((indName, oldAcc, newAcc))
#     return accs
    
# results = Parallel(n_jobs=14)(delayed(generateCombAccuracies)(oldColPairs, i) for i in tqdm(range(1,len(oldColPairs)+1)))
# print(results)

In [79]:
results_flat = [item for sublist in results for item in sublist[-1]]

In [ ]:
pd.Series(results_flat).describe()

In [93]:
results_flat = [item for sublist in results for item in sublist]

In [94]:
allCombs = (pd.DataFrame(results_flat, columns = ['Combination','MSE']).set_index('Combination')).reset_index()

In [95]:
# allCombs['Increase'] = allCombs['Accuracy (in %) of new embeddings compared to annotated category'] - allCombs['Accuracy (in %) of old embeddings compared to annotated category']

In [96]:
allCombs['count'] = allCombs['Combination'].apply(lambda p: p.count("&")+1)

In [97]:
# allCombs.to_csv('../data/wordsim353_all_combinations_SVM_accuracies.csv', index=None)

In [98]:
# allCombs = pd.read_csv('../data/wordsim353_all_combinations_SVM_accuracies.csv')

In [99]:
allCombs.head()

,Combination,MSE,count
0,Complex,0.023219,1
1,Transe,0.026213,1
2,Text,0.022456,1
3,Class,0.019362,1
4,JC_Sim,0.019337,1


In [100]:
print(f"Determined accuracies of {len(allCombs)} combinations of embedding scores based on the length {len(colPairs)}")

Determined accuracies of 31 combinations of embedding scores based on the length 5


In [101]:
pd.options.display.max_colwidth = 0

In [102]:
allCombs.sort_values(by=['MSE']).head(10)

,Combination,MSE,count
30,Complex & Transe & Text & Class & JC_Sim,0.016446,5
29,Transe & Text & Class & JC_Sim,0.016517,4
25,Complex & Transe & Text & Class,0.016782,4
28,Complex & Text & Class & JC_Sim,0.016802,4
21,Transe & Text & Class,0.016909,3
24,Text & Class & JC_Sim,0.016925,3
18,Complex & Text & Class,0.016934,3
19,Complex & Text & JC_Sim,0.017272,3
13,Text & JC_Sim,0.017273,2
12,Text & Class,0.017472,2


In [103]:
# allCombs.sort_values(by=['Accuracy (in %) of new embeddings compared to annotated category'], ascending=False).head(10)

In [104]:
# allCombs.sort_values(by=['Increase'], ascending=False).head(10)

In [105]:
allCombs[allCombs['count'] == 1].sort_values(by=['MSE']).head(10)

,Combination,MSE,count
4,JC_Sim,0.019337,1
3,Class,0.019362,1
2,Text,0.022456,1
0,Complex,0.023219,1
1,Transe,0.026213,1


## Classification

In [89]:
def labelSamples(score):
    return 'I' if score <= 1.75 else 'U' if score >= 3.5 else 'M'

In [ ]:
wordSim353AnnotDF_New_Merged_DF['category'] = wordSim353AnnotDF_New_Merged_DF['Avg']

In [88]:
from joblib import Parallel, delayed
def generateCombAccuraciesClassn(oldColPairs, r):
    accs = []
    for comb in tqdm(combinations(oldColPairs, r)):
        assert len(comb) == r
        oldAcc = trainAndFindAccuracyClassification(wordSim353AnnotDF_New_Merged_DF, [col[0] for col in comb])
#         newAcc = trainAndFindAccuracy(wordSim353AnnotDF_New_Merged_DF, [colMappers[col[0]] for col in comb])
        indName = " & ".join([col[1] for col in comb])
        accs.append((indName, oldAcc))
    return accs
    
resultsClassn = Parallel(n_jobs=14)(delayed(generateCombAccuraciesClassn)(colPairs, i) for i in tqdm(range(1,len(colPairs)+1)))
print(resultsClassn)

  0%|          | 0/5 [00:00<?, ?it/s]

KeyError: 'category'

In [ ]:
results_flat_classn = [item for sublist in resultsClassn for item in sublist]

In [ ]:
allCombs_classn = (pd.DataFrame(results_flat_classn, columns = ['Combination','MSE']).set_index('Combination')).reset_index()

In [ ]:
# allCombs['Increase'] = allCombs['Accuracy (in %) of new embeddings compared to annotated category'] - allCombs['Accuracy (in %) of old embeddings compared to annotated category']

In [79]:
allCombs_classn['count'] = allCombs_classn['Combination'].apply(lambda p: p.count("&")+1)

In [80]:
# allCombs.to_csv('../data/wordsim353_all_combinations_SVM_accuracies.csv', index=None)

In [81]:
# allCombs = pd.read_csv('../data/wordsim353_all_combinations_SVM_accuracies.csv')

In [82]:
allCombs_classn.head()

,Combination,MSE,count
0,Complex,3.064231,1
1,Transe,3.393020,1
2,Text,2.593313,1
3,Class,2.863816,1
4,JC_Sim,2.962828,1


In [83]:
print(f"Determined accuracies of {len(allCombs_classn)} combinations of embedding scores based on the length {len(colPairs)}")

Determined accuracies of 31 combinations of embedding scores based on the length 5


In [84]:
pd.options.display.max_colwidth = 0

In [85]:
allCombs_classn.sort_values(by=['MSE']).head(10)

,Combination,MSE,count
24,Text & Class & JC_Sim,2.337720,3
12,Text & Class,2.339691,2
13,Text & JC_Sim,2.369431,2
29,Transe & Text & Class & JC_Sim,2.411251,4
28,Complex & Text & Class & JC_Sim,2.415421,4
18,Complex & Text & Class,2.439534,3
21,Transe & Text & Class,2.473388,3
19,Complex & Text & JC_Sim,2.475702,3
22,Transe & Text & JC_Sim,2.491334,3
30,Complex & Transe & Text & Class & JC_Sim,2.497628,5


In [69]:
# allCombs.sort_values(by=['Accuracy (in %) of new embeddings compared to annotated category'], ascending=False).head(10)

In [70]:
# allCombs.sort_values(by=['Increase'], ascending=False).head(10)

In [72]:
allCombs_classn[allCombs_classn['count'] == 1].sort_values(by=['MSE']).head(10)

,Combination,MSE,count
2,Text,2.593313,1
3,Class,2.863816,1
4,JC_Sim,2.962828,1
0,Complex,3.064231,1
1,Transe,3.393020,1


## RF

In [250]:
from joblib import Parallel, delayed
def generateCombAccuracies(oldColPairs, r):
    accs = []
    for comb in tqdm(combinations(oldColPairs, r)):
        assert len(comb) == r
        oldAcc = trainAndFindAccuracyRF(wordSim353AnnotDF_New_Merged_DF, [col[0] for col in comb])
        newAcc = trainAndFindAccuracyRF(wordSim353AnnotDF_New_Merged_DF, [colMappers[col[0]] for col in comb])
        indName = " & ".join([col[1] for col in comb])
        accs.append((indName, oldAcc, newAcc))
    return accs
    
results1 = Parallel(n_jobs=14)(delayed(generateCombAccuracies)(oldColPairs, i) for i in tqdm(range(1,len(oldColPairs)+1)))
print(results1)

  0%|          | 0/14 [00:00<?, ?it/s]

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [263]:
results == results1

True

In [262]:
results_flat1 = [item for sublist in results1 for item in sublist]

In [252]:
allCombs1 = (pd.DataFrame(results_flat1, columns = ['Combination','Accuracy (in %) of old embeddings compared to annotated category', 'Accuracy (in %) of new embeddings compared to annotated category']).set_index('Combination') * 100).reset_index()

In [253]:
allCombs1['Increase'] = allCombs1['Accuracy (in %) of new embeddings compared to annotated category'] - allCombs['Accuracy (in %) of old embeddings compared to annotated category']

In [254]:
allCombs1['count'] = allCombs1['Combination'].apply(lambda p: p.count("&")+1)

In [255]:
allCombs1.head()

,Combination,Accuracy (in %) of old embeddings compared to annotated category,Accuracy (in %) of new embeddings compared to annotated category,Increase,count
0,Text 7 props - 19k,65.126050,63.672269,-1.453782,1
1,Text 2 props - 19k,61.899160,63.344538,1.445378,1
2,Complex - 19k,66.613445,64.277311,-2.336134,1
3,Transe - 19k,66.613445,66.310924,-0.302521,1
4,Abstract - 19k,68.899160,66.008403,-2.890756,1


In [256]:
print(f"Determined accuracies of {len(allCombs1)} combinations of embedding scores based on the length {len(oldColPairs)}")

Determined accuracies of 16383 combinations of embedding scores based on the length 14


In [257]:
pd.options.display.max_colwidth = 0

In [258]:
allCombs1.sort_values(by=['Accuracy (in %) of old embeddings compared to annotated category'], ascending=False).head(10)

,Combination,Accuracy (in %) of old embeddings compared to annotated category,Accuracy (in %) of new embeddings compared to annotated category,Increase,count
5,Abstract First Sentence - 19k,73.285714,70.075630,-3.210084,1
60,Abstract - 19k & Abstract First Sentence - 19k,72.117647,68.327731,-3.789916,2
138,Text 7 props - 19k & Abstract - 19k & Abstract First Sentence - 19k,72.100840,70.361345,-1.739496,3
151,Text 7 props - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - 19k,71.831933,70.378151,-1.453782,3
152,Text 7 props - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - Probase-19k,71.831933,71.252101,-0.579832,3
2114,Text 7 props - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k & 6 embeddings - Probase-19k,71.815126,69.789916,-2.025210,5
2001,Text 7 props - 19k & Abstract - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - Probase-19k,71.815126,69.798319,-2.016807,5
2000,Text 7 props - 19k & Abstract - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k,71.815126,70.378151,-1.436975,5
1999,Text 7 props - 19k & Abstract - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - 19k & 6 embeddings - Probase-19k,71.815126,69.495798,-2.319328,5
1998,Text 7 props - 19k & Abstract - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - 19k & 6 embeddings - 19k,71.815126,69.487395,-2.327731,5


In [259]:
allCombs1.sort_values(by=['Accuracy (in %) of new embeddings compared to annotated category'], ascending=False).head(10)

,Combination,Accuracy (in %) of old embeddings compared to annotated category,Accuracy (in %) of new embeddings compared to annotated category,Increase,count
681,Text 7 props - 19k & Abstract First Sentence - 19k & Transe - Probase & Complex-Transe-AbsFirstSent - Probase-19k,70.663866,71.260504,0.596639,4
218,Text 2 props - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - Probase-19k,71.226891,71.252101,0.025210,3
1575,Text 7 props - 19k & Text 2 props - 19k & Abstract - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - Probase-19k,71.235294,71.252101,0.016807,5
152,Text 7 props - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - Probase-19k,71.831933,71.252101,-0.579832,3
3884,Text 7 props - 19k & Text 2 props - 19k & Abstract First Sentence - 19k & Complex - Probase-19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k,68.310924,71.235294,2.924370,6
129,Text 7 props - 19k & Transe - 19k & Abstract First Sentence - 19k,69.764706,71.235294,1.470588,3
217,Text 2 props - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - 19k,71.226891,70.974790,-0.252101,3
604,Text 7 props - 19k & Transe - 19k & Abstract First Sentence - 19k & Complex-Transe-AbsFirstSent - Probase-19k,71.529412,70.966387,-0.563025,4
7207,Text 7 props - 19k & Text 2 props - 19k & Abstract First Sentence - 19k & Transe - Probase-19k & Complex-Transe-AbsFirstSent - 19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - Probase-19k,68.605042,70.966387,2.361345,7
690,Text 7 props - 19k & Abstract First Sentence - 19k & Transe - Probase-19k & Complex-Transe-AbsFirstSent - Probase-19k,70.663866,70.966387,0.302521,4


In [260]:
allCombs1.sort_values(by=['Increase'], ascending=False).head(10)

,Combination,Accuracy (in %) of old embeddings compared to annotated category,Accuracy (in %) of new embeddings compared to annotated category,Increase,count
13048,Text 7 props - 19k & Text 2 props - 19k & Complex - 19k & Transe - 19k & Abstract First Sentence - 19k & Complex - Probase & Transe - Probase & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k,64.252101,70.352941,6.100840,9
13084,Text 7 props - 19k & Text 2 props - 19k & Complex - 19k & Transe - 19k & Abstract First Sentence - 19k & Transe - Probase & Transe - Probase-19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k,64.252101,69.210084,4.957983,9
14022,Text 7 props - 19k & Complex - 19k & Abstract First Sentence - 19k & Transe - Probase & Transe - Probase-19k & Complex-Transe-AbsFirstSent - 19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k & 6 embeddings - Probase-19k,64.529412,69.176471,4.647059,9
13861,Text 7 props - 19k & Complex - 19k & Transe - 19k & Abstract First Sentence - 19k & Complex - Probase & Transe - Probase-19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k & 6 embeddings - Probase-19k,64.243697,68.890756,4.647059,9
7551,Text 7 props - 19k & Complex - 19k & Abstract - 19k & Complex - Probase & Complex - Probase-19k & Transe - Probase-19k & 6 embeddings - Probase-19k,63.092437,67.722689,4.630252,7
13058,Text 7 props - 19k & Text 2 props - 19k & Complex - 19k & Transe - 19k & Abstract First Sentence - 19k & Complex - Probase & Complex - Probase-19k & Complex-Transe-AbsFirstSent - Probase-19k & 6 embeddings - 19k,64.252101,68.605042,4.352941,9
6500,Text 7 props - 19k & Text 2 props - 19k & Complex - 19k & Transe - 19k & Abstract - 19k & Complex - Probase-19k & 6 embeddings - Probase-19k,63.689076,68.033613,4.344538,7
7537,Text 7 props - 19k & Complex - 19k & Abstract - 19k & Complex - Probase & Transe - Probase & Complex - Probase-19k & 6 embeddings - Probase-19k,63.092437,67.436975,4.344538,7
7570,Text 7 props - 19k & Complex - 19k & Abstract - 19k & Transe - Probase & Complex - Probase-19k & Transe - Probase-19k & 6 embeddings - 19k,63.092437,67.428571,4.336134,7
6499,Text 7 props - 19k & Text 2 props - 19k & Complex - 19k & Transe - 19k & Abstract - 19k & Complex - Probase-19k & 6 embeddings - 19k,63.689076,68.025210,4.336134,7


In [261]:
allCombs1[allCombs1['count'] == 1].sort_values(by=['Accuracy (in %) of new embeddings compared to annotated category'], ascending=False).head(10)

,Combination,Accuracy (in %) of old embeddings compared to annotated category,Accuracy (in %) of new embeddings compared to annotated category,Increase,count
11,Complex-Transe-AbsFirstSent - Probase-19k,70.042017,70.630252,0.588235,1
5,Abstract First Sentence - 19k,73.285714,70.075630,-3.210084,1
10,Complex-Transe-AbsFirstSent - 19k,70.042017,68.302521,-1.739496,1
13,6 embeddings - Probase-19k,68.907563,68.042017,-0.865546,1
12,6 embeddings - 19k,68.907563,66.857143,-2.050420,1
6,Complex - Probase,66.613445,66.596639,-0.016807,1
7,Transe - Probase,66.613445,66.596639,-0.016807,1
3,Transe - 19k,66.613445,66.310924,-0.302521,1
9,Transe - Probase-19k,66.613445,66.008403,-0.605042,1
4,Abstract - 19k,68.899160,66.008403,-2.890756,1


# Generate Embeddings Files

In [133]:
wordList = pd.concat([wordSim353AnnotDF_New2[['word1_kg_id', 'Word 1']], wordSim353AnnotDF_New2[['word2_kg_id', 'Word 2']].rename(columns={'Word 2': 'Word 1', 'word2_kg_id': 'word1_kg_id'})])

In [134]:
wordList = wordList[~wordList.duplicated()]

In [135]:
wordList['word1_kg_id'].apply(lambda p: "\t".join([str(p1) for p1 in embedDict[p].tolist()])).to_csv('../data/wordsim353_embeddings.tsv', index=False, header=None)

In [136]:
wordList['word1_kg_id'].apply(lambda p: "\t".join([str(p1) for p1 in newEmbedDict[p].tolist()])).to_csv('../data/wordsim353_embeddings_new_attempt1.tsv', index=False, header=None)

In [137]:
wordList['Word 1'].to_csv('../data/wordsim353_embeddings_words.tsv', index=False, header=None)